# a short script to make Athena queries quickly 

In [1]:
# make query to get cic data
def make_query(cic_dates, keys=[]):
    '''
    Make a query to get cic data from athena.
    INPUT:
        cic_dates: list of dicts containing cic id, min_date and max_date
            example cic_dates = [{'cic_id': 'CIC-008396fd-1fe4-5812-b410-91ac000e439c', 'min_date': '2022/12/20', 'max_date': '2022/12/20'}, {...}...]
        keys: list of keys to get from athena, list of strings example: ['system.quattBuild', 'system.isLteConnected']
    OUPUT:
        query_string: string containing the query to get data from athena
    '''

    query_string = ''

    for cic_date in cic_dates:
        try:
            cic = cic_date['cic_id']
            min_date = cic_date['min_date']
            max_date = cic_date['max_date']

            # query_string += '''SELECT now as time, clientid as cicid,'''
            query_string += '''SELECT arbitrary(clientid) as cicid, max(now) as time, '''
            for key in keys:
                query_string += f'''json_extract_scalar(payload, '$.stats[0].{key.lower()}') as {key.replace('.', '_')}, '''
            query_string = query_string[:-2]
            query_string += f''' FROM raw_iot_data WHERE cic_id = '{cic}' AND date_format >= '{min_date}' AND date_format <= '{max_date}' '''
            # query_string += ' AND json_extract_scalar(payload, \'$.stats[0].system.iswifireachable\') = \'true\' '
        except:
            raise ValueError('cic_dates must be a list of dicts containing cic_id, min_date and max_date')
        
        if cic_date != cic_dates[-1]:
            query_string += '\nUNION ALL\n'
        else :
            query_string += ';'
    return query_string